In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install statsmodels
!pip install xgboost
!pip install torchmetrics
!pip install piq
!pip install torch-fidelity

In [24]:
import numpy as np
import pandas as pd
from decimal import getcontext, Decimal
import cv2
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
import torch
import torch.nn.functional as F
from torchvision import transforms
import torchmetrics
import piq
from PIL import Image
from skimage.metrics import mean_squared_error as skimage_mse, peak_signal_noise_ratio as skimage_psnr, structural_similarity as skimage_ssim
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tools.eval_measures import mse, rmse
import statsmodels.api as sm

# Data

In [6]:
np.set_printoptions(precision=25)
getcontext().prec = 25
pd.options.display.float_format = '{:.25f}'.format
np.set_printoptions(precision=25)
pd.set_option('display.float_format', '{:.25f}'.format)

In [11]:
# Random 128*128
np.random.seed(42)
y_true = np.random.rand(128, 128).astype(np.float32)
y_pred = np.random.rand(128, 128).astype(np.float32)

In [8]:
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()
results = {}

#**Tensorflow Library:**

In [14]:
y_true_tf = tf.convert_to_tensor(y_true)
y_pred_tf= tf.convert_to_tensor(y_pred)

y_true_tf = tf.expand_dims(y_true, axis=-1)
y_pred_tf = tf.expand_dims(y_pred, axis=-1)
y_true_flat_tf = tf.convert_to_tensor(y_true_flat, dtype=tf.float32)
y_pred_flat_tf = tf.convert_to_tensor(y_pred_flat, dtype=tf.float32)

ssim_value_tf = tf.image.ssim(y_true_tf, y_pred_tf, max_val=1.0).numpy()
psnr_value_tf = tf.image.psnr(y_true_tf, y_pred_tf, max_val=1.0).numpy()

results['TensorFlow'] = {
    'SSIM': ssim_value_tf,
    'PSNR': psnr_value_tf
}

# **PyTorch Library:**

In [26]:
y_true_torch = torch.tensor(y_true).unsqueeze(0).unsqueeze(0)
y_pred_torch = torch.tensor(y_pred).unsqueeze(0).unsqueeze(0)

y_true_flat_tensor = torch.tensor(y_true_flat, dtype=torch.float32)
y_pred_flat_tensor = torch.tensor(y_pred_flat, dtype=torch.float32)

mae_metric = torchmetrics.MeanAbsoluteError()
mse_metric = torchmetrics.MeanSquaredError()
rmse_metric = torchmetrics.MeanSquaredError(squared=False)
r2_metric = torchmetrics.R2Score()

pt_mae = mae_metric(y_pred_flat_tensor, y_true_flat_tensor).item()
pt_mse = mse_metric(y_pred_flat_tensor, y_true_flat_tensor).item()
pt_rmse = rmse_metric(y_pred_flat_tensor, y_true_flat_tensor).item()
pt_r2 = r2_metric(y_pred_flat_tensor, y_true_flat_tensor).item()
ssim_value_torch = piq.ssim(y_true_torch, y_pred_torch, data_range=1.0).item()
psnr_value_torch = piq.psnr(y_true_torch, y_pred_torch, data_range=1.0).item()

results['PyTorch/torchmetrics'] = {
    'MAE': pt_mae,
    'MSE': pt_mse,
    'RMSE': pt_rmse,
    'SSIM': ssim_value_torch,
    'PSNR': psnr_value_torch,
    'R²': pt_r2,
}

# **Scikit-Image Library:**

In [27]:
skimage_mse_value = skimage_mse(y_true, y_pred)
skimage_psnr_value = skimage_psnr(y_true, y_pred)
L = float(y_true.max() - y_true.min())
skimage_ssim_value = skimage_ssim(y_true, y_pred, data_range = L)

results['Scikit-Image'] = {
    'MSE': skimage_mse_value,
    'PSNR': skimage_psnr_value,
    'SSIM': skimage_ssim_value
}

# **Scikit-Learn Library:**

In [ ]:
sklearn_mae = mean_absolute_error(y_true_flat, y_pred_flat)
sklearn_mse = mean_squared_error(y_true_flat, y_pred_flat)
sklearn_rmse = mean_squared_error(y_true_flat, y_pred_flat, squared=False)
sklearn_r2 = r2_score(y_true_flat, y_pred_flat)

results['Scikit-Learn'] = {
    'MAE': sklearn_mae,
    'MSE': sklearn_mse,
    'RMSE': sklearn_rmse,
    'R²': sklearn_r2
}

# **Statsmodels Library:**

In [29]:
model = sm.OLS(y_true_flat, sm.add_constant(y_pred_flat)).fit()
sm_mae = mean_absolute_error(y_true_flat, y_pred_flat)
sm_mse = mse(y_true_flat, y_pred_flat)
sm_rmse = rmse(y_true_flat, y_pred_flat)
sm_r2 = model.rsquared

results['Statsmodels'] = {
    'R²': sm_r2,
    'MAE': sm_mae,
    'MSE': sm_mse,
    'RMSE': sm_rmse
}

# **OpenCv Library:**

In [41]:

y_true_uint8 = (y_true * 255).astype('uint8') if y_true.max() <= 1 else y_true.astype('uint8')
y_pred_uint8 = (y_pred * 255).astype('uint8') if y_pred.max() <= 1 else y_pred.astype('uint8')
psnr_value_opencv = cv2.PSNR(y_true_uint8, y_pred_uint8)
ssim_value = ssim(y_true_uint8, y_pred_uint8, data_range=255)
ssim_index, _ = cv2.quality.QualitySSIM_compute(y_true_uint8, y_pred_uint8)
ssim_value = ssim_index[0]

results['OpenCv'] = {
    'PSNR': psnr_value_opencv,
    'SSIM': ssim_value,
}

In [ ]:
results_df = pd.DataFrame(results).T
results_df.reset_index(inplace=True)
results_df = results_df.rename_axis(None, axis=1)
print(results_df)